In [66]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
import time

In [98]:
def segador(url):
    enlaces = []
    active_url = url
    flag = -1
    while (flag != 0):
        time.sleep(5)
        next_page = None
        r2 = requests.get(active_url)
        print(active_url, r2.status_code)
        soup = BeautifulSoup(r2.text)
        for link in soup.find_all('a'):
            enlace = link.get('href')
            clases = link.get('class')
            if clases and 'prefetch' in clases:
                next_page = enlace
            if enlace.find('/MLM') != -1:
                enlaces.append(enlace)
        if next_page:
            active_url = next_page
            flag = 1
        else:
            flag = 0
    return list(set(enlaces))

In [68]:
test = get_links('https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/xochimilco/')

https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/xochimilco/ 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/xochimilco/_Desde_49 200


In [69]:
len(test)

56

In [70]:
def checador(url):
    r = requests.get(url)
    status = r.status_code
    if status == 200:
        soup = BeautifulSoup(r.text)
        resultado = int(soup.find(attrs={'class':'quantity-results'}).string.strip().split(" ")[0].replace(",",""))
        return resultado
    return 0

In [71]:
checador('https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/_PriceRange_0-1000000000000')

23562

In [72]:
base = 'https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/_PriceRange_'
ini = 0
incremento = 100000
fin = 1000000000
url = base + str(ini) + '-' + str(ini + incremento)
url

'https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/_PriceRange_0-100000'

In [73]:
checador(url)

84

In [105]:
def medidor(base,ini,fin,limite,incremento):
    mult = 0
    fin_temp = ini
    resultados = 0
    while (resultados < limite) or (fin_temp > fin):
        fin_temp = ini + incremento*mult
        resultados_temp = resultados
        if (mult % 50 == 0):
            print(fin_temp,mult,resultados_temp,incremento)
        mult = mult + 1
        test = str(ini) + '-' + str(ini + incremento*mult)
        url = base + test
        resultados = checador(url)
    return (fin_temp,resultados_temp)

In [104]:
medidor(base,0,1000000000,2000,100000)

0 0 0
1000000 10 19
2000000 20 65
3000000 30 149
4000000 40 354
5000000 50 523
6000000 60 647
7000000 70 787
8000000 80 881
9000000 90 989
10000000 100 1080
11000000 110 1148
12000000 120 1240
13000000 130 1341
14000000 140 1430
15000000 150 1513
16000000 160 1609
17000000 170 1675
18000000 180 1764
19000000 190 1817
20000000 200 1844
21000000 210 1861
22000000 220 1895
23000000 230 1914
24000000 240 1927
25000000 250 1950
26000000 260 1962
27000000 270 1984
28000000 280 1991
29000000 290 1998


(29000000, 1998)

In [97]:
checador('https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/')

24068

In [110]:
urbase = 'https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/'
total_final = checador(urbase)
print("total final: " + str(total_final))
base = urbase + '_PriceRange_'
ini = 0
fin = 1000000000
limite = 2000
total = 0
incremento_fix = 100000000
incremento = incremento_fix
enlaces = []
while (total + limite) < total_final:
    (fin_temp,resultados_temp) = medidor(base,ini,fin,limite,incremento)
    if resultados_temp != 0:
        print(ini,fin_temp,resultados_temp,total)
        url = base + str(ini) + '-' + str(fin_temp)
        enlaces_temp = segador(url)
        enlaces = enlaces + enlaces_temp
        ini = fin_temp
        total = total + resultados_temp
        if incremento != incremento_fix:
            incremento = incremento_fix
    else:
        incremento = int(np.floor(incremento * 0.5))
url = base + str(ini) + '-' + str(fin)
enlaces_temp = segador(url)
enlaces = enlaces + enlaces_temp

total final: 2253
0 0 0
0 0 0
0 0 0
0 25000000 1951 0
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_Desde_49_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_Desde_97_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_Desde_145_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_Desde_193_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_Desde_241_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.com/departamentos/venta/distrito-federal/cuajimalpa-de-morelos/_Desde_289_PriceRange_0-25000000 200
https://inmuebles.metroscubicos.co

In [111]:
len(enlaces)

2260

In [112]:
with open('../data/procesados/cdmx_enlaces_terrenos.json', 'w') as fp:
    json.dump(enlaces, fp)

['https://departamento.metroscubicos.com/MLM-764298595-excelente-departamento-en-venta-en-cuajimalpa-_JM#position=35&type=item&tracking_id=c11bfb2a-2662-496e-8e96-749059dde7b2',
 'https://departamento.metroscubicos.com/MLM-785746482-atencion-inversionistas-_JM#position=5&type=item&tracking_id=7856bdae-9207-43a1-a658-ec9c6e9420bc',
 'https://departamento.metroscubicos.com/MLM-782110903-departamento-venta-bosques-de-las-lomas-_JM#position=46&type=item&tracking_id=e0ed565f-cde1-4f35-bbc6-ea764052e015',
 'https://departamento.metroscubicos.com/MLM-775264641-preventa-de-departamento-en-santa-fe-_JM#position=6&type=item&tracking_id=67a4ca56-ba12-41f2-83c5-caa4f0b3fb16',
 'https://departamento.metroscubicos.com/MLM-766692089-departamento-en-venta-en-bosques-de-las-lomas-_JM#position=34&type=item&tracking_id=b8157ff1-077d-4eff-bf28-ec279b4ea70e',
 'https://departamento.metroscubicos.com/MLM-776139668-departamento-en-preventa-san-mateo-tlaltenango-_JM#position=45&type=item&tracking_id=67a4ca56-